In [26]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2023-06-03 02:21:10--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2,6K) [application/zip]
Saving to: 'source.zip.3'

     0K ..                                                    100%  364M=0s

2023-06-03 02:21:11 (364 MB/s) - 'source.zip.3' saved [2707/2707]



In [27]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime
import zipfile

In [28]:
target_file = "C:\\Users\\Public\\ETL\\transformed_data.csv"
temp_file = "temp_data.tmp"
log_file = "log_file.txt"

with zipfile.ZipFile("source.zip", "r") as zip_ref:
    zip_ref.extractall(target_file)



In [29]:
def extract_csv(file_name):
    dataframe = pd.read_csv(file_name)
    return dataframe

def extract_json(file_name):
    dataframe = pd.read_json(file_name, lines=True)
    return dataframe

def extract_xml(file_name):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_name)
    root = tree.getroot()

    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)

    return dataframe


In [30]:
def extract():
    extracted_data = pd.DataFrame(columns=["name", "height", "weight"])

    for csv_file in glob.glob("*.csv"):
        extracted_data = pd.concat([extracted_data, extract_csv(csv_file)],ignore_index = True)

    for json_file in glob.glob("*.json"):
        extracted_data = pd.concat([extracted_data, extract_json(json_file)], ignore_index=True)

    for xml_file in glob.glob("*.xml"):
        extracted_data = pd.concat([extracted_data, extract_xml(xml_file)], ignore_index = True)


In [31]:
def transform(data):
    data["height"] = round(data.height * 0.025, 2)
    data["weight"] = round(data.weight * 0.454, 2)
    return data

In [32]:
def load(target_file, data_to_load):
    data_to_load.to_csv(target_file)


In [33]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(log_file, "a") as logfile:
        logfile.write(timestamp + ',' + message + '\n')

In [35]:
log("Extract phase started")
extracted_data = extract()
log("Extract phase ended")
extracted_data

log("Transformation phase started")
transformed_data = transform(extracted_data)
log("Transofrmation phase ended")
transformed_data

log("Load phase started")
load(target_file, transformed_data)
log("Load phase ended")

PermissionError: [Errno 13] Permission denied: 'transformed_data.csv'